# Convolutional Neural Networks for Image Recognition
## Advanced Machine Learning

In [1]:
!pip install pypng

/bin/bash: /home/aevans/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import (
    Convolution2D,
    Dense,
    MaxPooling2D,
    GlobalMaxPooling2D,
    Flatten,
    BatchNormalization,
    Dropout,
    SpatialDropout2D,
)

# from tensorflow.keras.optimizers import RMSprop
import random
import re

# From pypng
import png

# from sklearn.p
import sklearn.metrics

##################
# Configure figure parameters

FONTSIZE = 18
FIGURE_SIZE = (10, 4)
FIGURE_SIZE2 = (10, 10)

plt.rcParams.update({"font.size": FONTSIZE})
plt.rcParams["figure.figsize"] = FIGURE_SIZE
# Default tick label size
plt.rcParams["xtick.labelsize"] = FONTSIZE
plt.rcParams["ytick.labelsize"] = FONTSIZE

2023-05-30 17:51:40.704604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 17:51:40.836034: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-30 17:51:42.271604: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs:/home/aevans/miniconda3/lib/:/home/aevans/miniconda3/lib/
2023-05-30 17:51:42.271930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfe

In [3]:
# Load up a single image & view it
directory = "core50_128x128/s1/o21"
files = os.listdir(directory)

r = png.Reader(directory + "/" + files[0])
it = r.read()
image_2d = np.vstack(map(np.uint8, it[2]))
image_3d = np.reshape(image_2d, (128, 128, 3))

plt.imshow(image_3d)

FileNotFoundError: [Errno 2] No such file or directory: 'core50_128x128/s1/o21'

## Image Reading

In [ ]:
def readPngFile(filename):
    """
    Read a single PNG file

    filename = fully qualified file name

    Return: 3D numpy array (rows x cols x chans)

    Note: all pixel values are floats in the range 0.0 .. 1.0

    This implementation relies on the pypng package
    """
    # print("reading:", filename)
    # Load in the image meta-data
    r = png.Reader(filename)
    it = r.read()

    # Load in the image itself and convert to a 2D array
    image_2d = np.vstack(map(np.uint8, it[2]))

    # Reshape into rows x cols x chans
    image_3d = np.reshape(image_2d, (it[0], it[1], it[3]["planes"])) / 255.0
    return image_3d


def read_images_from_directory(directory, file_regexp):
    """
    Read a set of images from a directory.  All of the images must be the same size

    directory = Directory to search

    file_regexp = a regular expression to match the file names against

    Return: 4D numpy array (images x rows x cols x chans)
    """

    print(directory, file_regexp)
    # Get all of the file names
    files = sorted(os.listdir(directory))

    # Construct a list of images from those that match the regexp
    list_of_images = [
        readPngFile(directory + "/" + f) for f in files if re.search(file_regexp, f)
    ]

    # Create a 3D numpy array
    return np.array(list_of_images, dtype=np.float32)


def read_image_set_from_directories(directory, spec):
    """
    Read a set of images from a set of directories

    directory  = base directory to read from

    spec = n x 2 array of subdirs and file regexps

    Return: 4D numpy array (images x rows x cols x chans)

    """
    out = read_images_from_directory(directory + "/" + spec[0][0], spec[0][1])
    for sp in spec[1:]:
        out = np.append(
            out, read_images_from_directory(directory + "/" + sp[0], sp[1]), axis=0
        )
    return out


def load_multiple_image_sets_from_directories(
    directory_base, directory_list, object_list, test_files
):
    """ """
    print("##################")
    # Create the list of object/image specs
    inputs = [[obj, test_files] for obj in object_list]

    # First directory
    ret = read_image_set_from_directories(
        directory_base + "/" + directory_list[0], inputs
    )

    # Loop over directories
    for directory in directory_list[1:]:
        ret = np.append(
            ret,
            read_image_set_from_directories(directory_base + "/" + directory, inputs),
            axis=0,
        )

    return ret

## Load Data Sets

In [ ]:
## File location
directory_base = "core50_128x128"

# Training set: define which files to load for each object
# test_files = '.*[05].png'
test_files = ".*[0].png"

### Positive cases
# Define which objects to load
# object_list = ['o25', 'o22', 'o23', 'o24']
object_list = ["o21"]

# Define which conditions to load
# condition_list = ['s1', 's2', 's3', 's4', 's5', 's7', 's8', 's9', 's10', 's11']
# condition_list = ['s1', 's2', 's3', 's4']
condition_list = ["s1"]

# Load all of the objects/condition
ins_pos = load_multiple_image_sets_from_directories(
    directory_base, condition_list, object_list, test_files
)

### Negative cases
# Define which objects to load
# object_list2 = ['o45', 'o42', 'o43', 'o44']
object_list2 = ["o41"]
ins_neg = load_multiple_image_sets_from_directories(
    directory_base, condition_list, object_list2, test_files
)

### Combine positives and negatives into a common data set
outs_pos = np.append(
    np.ones((ins_pos.shape[0], 1)), np.zeros((ins_pos.shape[0], 1)), axis=1
)
outs_neg = np.append(
    np.zeros((ins_pos.shape[0], 1)), np.ones((ins_pos.shape[0], 1)), axis=1
)

ins = np.append(ins_pos, ins_neg, axis=0)
outs = np.append(outs_pos, outs_neg, axis=0)

########################################################################
# Validation set
# Define which files to load for each object
test_files = ".*[5].png"

### Positives
# Define which objects to load
object_list = ["o21"]
# object_list = ['o21']

# Load the positives
ins_pos_validation = load_multiple_image_sets_from_directories(
    directory_base, condition_list, object_list, test_files
)

### Negatives
# Define objects
object_list2 = ["o41"]
# object_list2 = ['o41']

# Load the negative cases
ins_neg_validation = load_multiple_image_sets_from_directories(
    directory_base, condition_list, object_list2, test_files
)

### Combine positives and negatives
outs_pos_validation = np.append(
    np.ones((ins_pos_validation.shape[0], 1)),
    np.zeros((ins_pos_validation.shape[0], 1)),
    axis=1,
)
outs_neg_validation = np.append(
    np.zeros((ins_pos_validation.shape[0], 1)),
    np.ones((ins_pos_validation.shape[0], 1)),
    axis=1,
)

ins_validation = np.append(ins_pos_validation, ins_neg_validation, axis=0)
outs_validation = np.append(outs_pos_validation, outs_neg_validation, axis=0)

In [ ]:
ins.shape

In [ ]:
outs.shape

In [ ]:
outs

In [ ]:
ins_validation.shape

In [ ]:
outs_validation.shape

## Create Network

In [ ]:
def create_classifier_network(
    image_size,
    nchannels,
    n_classes,
    learning_rate=0.0001,
    lambda_l2=0.0001,
    p_dropout=None,
    p_spatial_dropout=None,
    n_filters=[10],
    kernel_size=[3],
    pooling=[1],
    n_hidden=[5],
):
    model = Sequential()
    model.add(InputLayer(input_shape=(image_size[0], image_size[1], nchannels)))

    # convolutional layers
    for i, (n, s, p) in enumerate(zip(n_filters, kernel_size, pooling)):
        model.add(
            Convolution2D(
                filters=n,
                kernel_size=s,
                padding="same",
                use_bias=True,
                name="C%d" % (i),
                activation="elu",
            )
        )

        if p_spatial_dropout is not None:
            model.add(SpatialDropout2D(p_spatial_dropout))

        if p > 1:
            model.add(MaxPooling2D(pool_size=p, strides=p, name="MP%d" % (i)))

    # Flatten
    model.add(GlobalMaxPooling2D())

    # Dense layers
    for i, n in enumerate(n_hidden):
        model.add(Dense(units=n, activation="elu", use_bias="True", name="D%d" % i))

        if p_dropout is not None:
            model.add(Dropout(p_dropout))

    # Output
    model.add(
        Dense(units=n_classes, activation="softmax", use_bias="True", name="output")
    )

    # Optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=False)

    # Bind the model to the optimizer
    model.compile(
        loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy"]
    )

    print(model.summary())

    return model

In [ ]:
model = create_classifier_network(
    (ins.shape[1], ins.shape[2]),
    ins.shape[3],
    2,
    learning_rate=0.0001,
    p_spatial_dropout=0.1,
    # p_dropout=0.5,
    n_filters=[10, 20, 20, 40, 40, 80],
    kernel_size=[3, 3, 5, 5, 5, 5],
    pooling=[0, 2, 0, 2, 0, 2],
    n_hidden=[40, 20, 10, 5],
)

In [ ]:
# Callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True, min_delta=0.01
)

In [ ]:
history = model.fit(
    x=ins,
    y=outs,
    epochs=30,
    verbose=1,
    validation_data=(ins_validation, outs_validation),
    callbacks=[early_stopping_cb],
)

## Evaluation

In [ ]:
history.history.keys()

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label="Training")
plt.plot(history.history["val_loss"], label="Validation")
plt.legend()

In [ ]:
plt.figure()
plt.plot(history.history["categorical_accuracy"], label="Training")
plt.plot(history.history["val_categorical_accuracy"], label="Validation")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()

In [ ]:
ins_validation.shape

In [ ]:
np.argmax(model.predict(ins_validation), axis=1)

In [ ]:
outs_validation

In [ ]:
def generate_roc(model, ins, outs, ins_validation, outs_validation):
    """
    Produce a ROC plot given a model, a set of inputs and the true outputs

    Assume that model produces N-class output; we will only look at the class 0 scores
    """
    # Compute probabilistic predictions given images
    pred = model.predict(ins)
    # Compute false positive rate & true positive rate + AUC
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(outs[:, 0], pred[:, 0])
    auc = sklearn.metrics.auc(fpr, tpr)

    # Compute probabilistic predictions given images
    pred_val = model.predict(ins_validation)
    # Compute false positive rate & true positive rate + AUC
    fpr_val, tpr_val, thresholds_val = sklearn.metrics.roc_curve(
        outs_validation[:, 0], pred_val[:, 0]
    )
    auc_val = sklearn.metrics.auc(fpr_val, tpr_val)

    # Generate the plot
    plt.figure(1)
    plt.axis("equal")
    plt.plot([0, 1], [0, 1], "k--")
    plt.plot(fpr, tpr, "b", label="Train AUC = {:.3f}".format(auc))
    plt.plot(fpr_val, tpr_val, "r", label="Validation AUC = {:.3f}".format(auc_val))
    plt.legend(loc="best")
    plt.xlabel("FPR", fontsize=FONTSIZE)
    plt.ylabel("TPR", fontsize=FONTSIZE)

In [ ]:
generate_roc(model, ins, outs, ins_validation, outs_validation)

## Visualize Model Internals

In [ ]:
def intermediate_model_state(model, ins, layer_list):
    """
    Return layer activations for intermediate layers in a model for a set of examples

    :param model: Model in question
    :param ins: Input tensor (examples, rows, cols, channels)
    :param layer_list: List of layer names to produce activations for
    :returns: a list of numpy arrays
    """
    # Translate layer names into corresponding output tensors
    layer_outputs = [l.output for l in model.layers if l.name in layer_list]

    # Construct a new Keras model that outputs these tensors
    # The internal structure of the model itself is referenced through the input and output tensor lists
    new_model = keras.models.Model(inputs=model.input, outputs=layer_outputs)

    # Evaluate the new model
    activations = new_model.predict(ins)

    # Return a list of activation numpy arrays
    return activations


def visualize_state(activations, width=1, example=0, cmap="plasma"):
    """
    Produce graphical representation of a set of image channels

    :param activations: numpy array (example, rows, cols, channels)
    :param width: Number of images displayed horizontally
    :param example: Index of example to display
    :param cmap: Color map to use for plotting
    """
    # Size of the individual images
    nrows = activations.shape[1]
    ncols = activations.shape[2]
    # Number of channels
    nfilters = activations.shape[3]

    # Tile all of the sub-images
    grid = np.zeros((int((nfilters - 1) / width + 1) * nrows, ncols * width))

    # Loop over image
    for i in range(nfilters):
        # Compute r,c of tile to place the ith image into
        r = int(i / width)
        c = i % width
        grid[nrows * r : nrows * (r + 1), ncols * c : ncols * (c + 1)] = activations[
            example, :, :, i
        ]

    # Plot
    plt.matshow(grid, cmap=cmap)

In [ ]:
# Compute activations for 2 layers over a set of examples
layer_list = ["C5"]
activations = intermediate_model_state(model, ins_validation, layer_list)

In [ ]:
# Plot convolutional layers 1 and 2
example = 20
plt.imshow(ins_validation[example, :, :, :])
visualize_state(activations, width=10, example=example)
# visualize_state(activations[0], width=10, example=example)
# visualize_state(activations[1], width=20, example=example)
# visualize_state(activations[2], width=20, example=example)
# visualize_state(activations[3], width=30, example=example)

In [ ]:
# Plot convolutional layers 1 and 2
example = 40
plt.imshow(ins_validation[example, :, :, :])
visualize_state(activations, width=10, example=example)
# visualize_state(activations[0], width=10, example=example)
# visualize_state(activations[1], width=20, example=example)
# visualize_state(activations[2], width=20, example=example)
# visualize_state(activations[3], width=30, example=example)